In [1]:
!wget https://raw.githubusercontent.com/PacktPublishing/Transformers-for-Natural-Language-Processing/main/Chapter03/kant.txt

--2023-02-10 09:42:48--  https://raw.githubusercontent.com/PacktPublishing/Transformers-for-Natural-Language-Processing/main/Chapter03/kant.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11258065 (11M) [text/plain]
Saving to: ‘kant.txt’

kant.txt            100%[===================>]  10.74M  --.-KB/s    in 0.04s   

2023-02-10 09:42:49 (257 MB/s) - ‘kant.txt’ saved [11258065/11258065]



In [2]:
%%capture
!pip install transformers
!pip install tokenizers

In [3]:
%%time
from pathlib import Path 
from tokenizers import ByteLevelBPETokenizer
from tokenizers import BertWordPieceTokenizer
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
path = '/content/kant.txt'

In [5]:
%%time
tokenizer = ByteLevelBPETokenizer()
tokenizer.train(files=path,
                vocab_size = 52000,
                min_frequency = 2)

CPU times: user 7.41 s, sys: 184 ms, total: 7.59 s
Wall time: 8.13 s


In [6]:
tokenizer.encode('that is expreiemt one').tokens

['that', 'Ġis', 'Ġex', 'pre', 'i', 'em', 't', 'Ġone']

In [7]:
import os
token_dir = '/content/KantaiBERT-MLM'
if not os.path.exists(token_dir):
  os.makedirs(token_dir)

tokenizer.save_model('KantaiBERT-MLM')  

['KantaiBERT-MLM/vocab.json', 'KantaiBERT-MLM/merges.txt']

In [8]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

tokenizer = ByteLevelBPETokenizer(
    '/content/KantaiBERT-MLM/vocab.json',
'/content/KantaiBERT-MLM/merges.txt' )

then you take the things that you want

In [9]:
from transformers import RobertaConfig
config = RobertaConfig(
    vocab_size = 52000,
    max_position_embeddings = 514 ,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [10]:
config

RobertaConfig {
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 52000
}

i though that is a json file and i can edit it but it's a config type

In [11]:
tokenizer.encode("The Critique of Pure Reason.").tokens

['The', 'ĠCritique', 'Ġof', 'ĠPure', 'ĠReason', '.']

In [12]:
tokenizer.encode("The Critique of Pure Reason.").ids

[798, 2240, 265, 1405, 1265, 13]

In [13]:
from transformers import RobertaTokenizer
tokenizer = RobertaTokenizer.from_pretrained('/content/KantaiBERT-MLM',
                                             max_lwngth = 512)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [14]:
token = tokenizer("The Critique of Pure Reason.")['input_ids']

In [15]:
tokenizer.decode(token)

'<s> The Critique of Pure Reason. </s>'

In [16]:
from transformers import RobertaForMaskedLM
model = RobertaForMaskedLM(config)

In [17]:
model.config

RobertaConfig {
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 52000
}

In [18]:
print(model.num_parameters())

83504416


In [19]:
%%time
from transformers import LineByLineTextDataset
dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path = '/content/kant.txt',
    block_size = 128 
)

/usr/local/lib/python3.8/dist-packages/transformers/data/datasets/language_modeling.py:121: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


CPU times: user 32.8 s, sys: 417 ms, total: 33.2 s
Wall time: 37.5 s


In [20]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer = tokenizer,
    mlm = True , 
    mlm_probability = 0.15
)

In [21]:
from transformers import Trainer , TrainingArguments

model_name = "/content/KantaiBERT-MLM"


training_args = TrainingArguments(
  output_dir = model_name ,
  learning_rate = 2e-5 ,
  overwrite_output_dir= True,
  num_train_epochs = 6 ,
  per_device_train_batch_size=16,
  weight_decay=0.01,
  save_strategy="epoch",
  push_to_hub=True,
)

In [22]:
trainer = Trainer(
    model = model ,
    args = training_args ,
    data_collator = data_collator ,
    train_dataset = dataset ,
    tokenizer = tokenizer 
)

Cloning https://huggingface.co/mahmoudNG/KantaiBERT-MLM into local empty directory.


In [ ]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 170964
  Num Epochs = 6
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 64116
  Number of trainable parameters = 83504416


Step,Training Loss
500,7.451700
1000,6.294900
1500,6.112400
2000,6.021900
2500,5.914600
3000,5.793200
3500,5.750900
4000,5.547200
4500,5.438300
5000,5.283300


Saving model checkpoint to /content/KantaiBERT-MLM/checkpoint-10686
Configuration saved in /content/KantaiBERT-MLM/checkpoint-10686/config.json
Model weights saved in /content/KantaiBERT-MLM/checkpoint-10686/pytorch_model.bin
tokenizer config file saved in /content/KantaiBERT-MLM/checkpoint-10686/tokenizer_config.json
Special tokens file saved in /content/KantaiBERT-MLM/checkpoint-10686/special_tokens_map.json
added tokens file saved in /content/KantaiBERT-MLM/checkpoint-10686/added_tokens.json
tokenizer config file saved in /content/KantaiBERT-MLM/tokenizer_config.json
Special tokens file saved in /content/KantaiBERT-MLM/special_tokens_map.json
added tokens file saved in /content/KantaiBERT-MLM/added_tokens.json
Saving model checkpoint to /content/KantaiBERT-MLM/checkpoint-21372
Configuration saved in /content/KantaiBERT-MLM/checkpoint-21372/config.json
Model weights saved in /content/KantaiBERT-MLM/checkpoint-21372/pytorch_model.bin
tokenizer config file saved in /content/KantaiBERT-